#### Training Notebook - Histopathology Cancer Detection
This notebook trains a CNN for binary classification of histopathology images (cancerous vs. non-cancerous).
It uses PyTorch Lightning for training and MLflow for logging experiments.


#### Setup and Imports

In [ ]:
# Import necessary libraries
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from scripts.data_utils import HistopathologyDataModule
from scripts.model_utils import BaselineCNN
from scripts.config import BATCH_SIZE, LEARNING_RATE, EPOCHS

#### Initialize MLflow Logger

In [ ]:
# Setup MLflow Logger
mlflow_logger = MLFlowLogger(
    experiment_name="Histopathology Cancer Detection - Training",
    tracking_uri="file:./experiments/mlruns"
)

#### Initialize Trainer and Data Module
The trainer included early stopping, checkpointing, and MLflow logging. The data module handles loading, augmentation, and batching.

In [ ]:
# Define PyTorch Lightning Trainer with callbacks
trainer = pl.Trainer(
    max_epochs=EPOCHS,
    logger=mlflow_logger,
    gpus=1 if torch.cuda.is_available() else 0,
    callbacks=[
        ModelCheckpoint(
            monitor="val_loss",
            dirpath="checkpoints/",
            filename="best_model",
            save_top_k=1,
            mode="min"
        ),
        EarlyStopping(
            monitor="val_loss",
            patience=3,
            mode="min"
        )
    ]
)

#### Load data and model

In [ ]:
data_module = HistopathologyDataModule()
model = BaselineCNN()

#### Train the model
Train the model using the specified configuration and save the best checkpoint.

In [ ]:
trainer.fit(model, data_module)

In [ ]:
# Save the best model
best_model_path = trainer.checkpoint_callback.best_model_path
print(f"Best model saved at: {best_model_path}")

#### Log the Best Model Path to Mlflow and Sync with GitHub
Log the best model path to MLflow and ensure GitHub is synced to save experiment results.

In [ ]:
# Log the best model path and metrics to MLflow
mlflow_logger.experiment.log_param(
    run_id=mlflow_logger.run_id,
    key="best_model_path",
    value=best_model_path
)

In [ ]:
# Log key metrics to MLflow
metrics = trainer.callback_metrics
mlflow_logger.experiment.log_metrics(
    run_id=mlflow_logger.run_id,
    metrics={
        "train_accuracy": metrics["train_accuracy"].item(),
        "val_accuracy": metrics["val_accuracy"].item(),
        "val_precision": metrics["val_precision"].item(),
        "val_recall": metrics["val_recall"].item(),
        "val_f1": metrics["val_f1"].item(),
        "val_auc": metrics["val_auc"].item()
    }
)

#### Save checkpoint to GitHub if on Kaggle

In [ ]:
!git add checkpoints/best_model.ckpt
!git commit -m "Add best model checkpoint"
!git push origin main